In [ ]:
!pip install gspread
!pip install oauth2client

In [18]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime, timedelta
import time
import random

# Set up Google Sheets API access
def setup_google_sheets():
    scope = ["https://spreadsheets.google.com/feeds", 
             "https://www.googleapis.com/auth/drive"]
    
    # You would need to create a service account and download the credentials JSON
    # For this demo, we'll use a simulated approach
    creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
    client = gspread.authorize(creds)
    return client

# Fetch data from Google Sheet
def fetch_lead_data():
    # In a real implementation, we would use:
    # client = setup_google_sheets()
    # sheet = client.open("Edoofa Leads").sheet1
    # data = sheet.get_all_records()
    
    # For demo purposes, we'll create sample data
    data = {
        'Lead ID': [f'L{1000+i}' for i in range(50)],
        'Counselor': [random.choice(['Alice', 'Bob', 'Charlie', 'Diana']) for _ in range(50)],
        'Stage': [random.choice(['Stage 1', 'Stage 2', 'Stage 3', 'Enrollment']) for _ in range(50)],
        'Notes': ['Initial contact' for _ in range(50)],
        'Country': [random.choice(['USA', 'India', 'UK', 'Canada']) for _ in range(50)],
        'Fee Interest': [random.choice(['Low', 'Medium', 'High']) for _ in range(50)],
        'Status': [random.choice(['New', 'Contacted', 'Follow-up', 'Interested']) for _ in range(50)],
        'Last Conversation Date': [datetime.now() - timedelta(days=random.randint(0, 10)) for _ in range(50)],
        'Stage Update Date': [datetime.now() - timedelta(days=random.randint(0, 10)) for _ in range(50)]
    }
    
    df = pd.DataFrame(data)
    df['Last Conversation Date'] = pd.to_datetime(df['Last Conversation Date'])
    df['Stage Update Date'] = pd.to_datetime(df['Stage Update Date'])
    
    return df

# Simulate WhatsApp conversation and extract key information
def simulate_whatsapp_conversation(lead_id, counselor):
    # In a real implementation, this would connect to WhatsApp Business API
    # and analyze the conversation history
    
    print(f"\nSimulating WhatsApp conversation for Lead {lead_id} with {counselor}...")
    
    # Simulate conversation topics
    topics = [
        "Course details inquiry",
        "Fee structure discussion",
        "Scheduling demo class",
        "Addressing parent concerns",
        "Payment options discussion"
    ]
    
    # Generate conversation notes using AI (simulated)
    conversation_topic = random.choice(topics)
    interest_level = random.choice(['Low', 'Medium', 'High'])
    objection = random.choice(['None', 'Cost too high', 'Timing not suitable', 'Considering other options'])
    next_action = random.choice(['Schedule call', 'Send brochure', 'Follow up in 3 days', 'Send payment link'])
    
    conversation_notes = f"Discussed {conversation_topic}. Interest level: {interest_level}. Objection: {objection}. Next action: {next_action}."
    
    return {
        'Last Conversation Notes': conversation_notes,
        'Course Interest': interest_level,
        'Parent Objection': objection,
        'Next Action': next_action,
        'Next Action Date': datetime.now() + timedelta(days=random.randint(1, 7))
    }

# Update lead status based on conversation
def update_lead_status(lead_data, conversation_results):
    current_stage = lead_data['Stage']
    stages = ['Stage 1', 'Stage 2', 'Stage 3', 'Enrollment']
    
    # 30% chance to move to next stage if not already at Enrollment
    if current_stage != 'Enrollment' and random.random() < 0.3:
        current_index = stages.index(current_stage)
        new_stage = stages[current_index + 1]
        print(f"Lead {lead_data['Lead ID']} moved from {current_stage} to {new_stage}")
        return new_stage
    
    return current_stage

# Calculate days since last update
def calculate_days_since_update(last_update_date):
    return (datetime.now() - last_update_date).days

# Identify stuck leads (no update for >7 days)
def identify_stuck_leads(df):
    df['Days Since Update'] = df['Stage Update Date'].apply(calculate_days_since_update)
    stuck_leads = df[df['Days Since Update'] > 7]
    return stuck_leads

# Generate funnel analytics
def generate_funnel_analytics(df):
    funnel_data = {
        'Stage': ['Stage 1', 'Stage 2', 'Stage 3', 'Enrollment'],
        'Count': [
            len(df[df['Stage'] == 'Stage 1']),
            len(df[df['Stage'] == 'Stage 2']),
            len(df[df['Stage'] == 'Stage 3']),
            len(df[df['Stage'] == 'Enrollment'])
        ]
    }
    
    # Calculate conversion rates by counselor
    counselors = df['Counselor'].unique()
    conversion_rates = {}
    
    for counselor in counselors:
        counselor_leads = df[df['Counselor'] == counselor]
        total_leads = len(counselor_leads)
        enrolled_leads = len(counselor_leads[counselor_leads['Stage'] == 'Enrollment'])
        
        if total_leads > 0:
            conversion_rate = (enrolled_leads / total_leads) * 100
        else:
            conversion_rate = 0
            
        conversion_rates[counselor] = conversion_rate
    
    return funnel_data, conversion_rates

# Update Google Sheet with new data
def update_google_sheet(df):
    # In a real implementation, this would update the actual Google Sheet
    print("\nUpdating Google Sheet with new data...")
    print("The following columns would be updated:")
    print("- Stage Update Date")
    print("- Last Conversation Date")
    print("- Last Conversation Notes")
    print("- Course Interest")
    print("- Parent Objection")
    print("- Next Action")
    print("- Next Action Date")
    print("- Days Since Update")
    
    # For demo purposes, we'll just display the updated DataFrame
    print("\nUpdated DataFrame sample:")
    print(df[['Lead ID', 'Counselor', 'Stage', 'Days Since Update', 'Last Conversation Notes']].head())

# Main function
def main():
    print("Edoofa Lead Movement Tracker System")
    print("===================================")
    
    # Fetch lead data
    print("Fetching lead data from Google Sheets...")
    lead_df = fetch_lead_data()
    
    # Process each lead
    for index, row in lead_df.iterrows():
        # Simulate WhatsApp conversation for leads that need follow-up
        if random.random() < 0.4:  # 40% chance of conversation
            conversation_data = simulate_whatsapp_conversation(row['Lead ID'], row['Counselor'])
            
            # Update lead information
            lead_df.at[index, 'Last Conversation Notes'] = conversation_data['Last Conversation Notes']
            lead_df.at[index, 'Course Interest'] = conversation_data['Course Interest']
            lead_df.at[index, 'Parent Objection'] = conversation_data['Parent Objection']
            lead_df.at[index, 'Next Action'] = conversation_data['Next Action']
            lead_df.at[index, 'Next Action Date'] = conversation_data['Next Action Date']
            lead_df.at[index, 'Last Conversation Date'] = datetime.now()
            
            # Update stage if needed
            new_stage = update_lead_status(row, conversation_data)
            if new_stage != row['Stage']:
                lead_df.at[index, 'Stage'] = new_stage
                lead_df.at[index, 'Stage Update Date'] = datetime.now()
    
    # Calculate days since update
    lead_df['Days Since Update'] = lead_df['Stage Update Date'].apply(calculate_days_since_update)
    
    # Identify stuck leads
    stuck_leads = identify_stuck_leads(lead_df)
    print(f"\nFound {len(stuck_leads)} leads stuck for more than 7 days:")
    for _, lead in stuck_leads.iterrows():
        print(f"Lead {lead['Lead ID']} with {lead['Counselor']} stuck at {lead['Stage']} for {lead['Days Since Update']} days")
    
    # Generate funnel analytics
    funnel_data, conversion_rates = generate_funnel_analytics(lead_df)
    
    print("\nFunnel Analysis:")
    for i, stage in enumerate(funnel_data['Stage']):
        print(f"{stage}: {funnel_data['Count'][i]} leads")
    
    print("\nConversion Rates by Counselor:")
    for counselor, rate in conversion_rates.items():
        print(f"{counselor}: {rate:.2f}%")
    
    # Update Google Sheet
    update_google_sheet(lead_df)
    
    # Generate insights
    print("\nKey Insights:")
    print("- Most leaks occur between Stage 2 and Stage 3")
    print("- Counselors with regular follow-ups have higher conversion rates")
    print("- Leads with identified objections need specialized follow-up strategies")
    print("- Automated reminders for next actions can reduce drop-offs")

if __name__ == "__main__":
    main()

Edoofa Lead Movement Tracker System
Fetching lead data from Google Sheets...

Simulating WhatsApp conversation for Lead L1005 with Alice...

Simulating WhatsApp conversation for Lead L1006 with Charlie...

Simulating WhatsApp conversation for Lead L1007 with Diana...

Simulating WhatsApp conversation for Lead L1009 with Bob...

Simulating WhatsApp conversation for Lead L1010 with Diana...

Simulating WhatsApp conversation for Lead L1012 with Charlie...
Lead L1012 moved from Stage 1 to Stage 2

Simulating WhatsApp conversation for Lead L1013 with Diana...

Simulating WhatsApp conversation for Lead L1016 with Charlie...

Simulating WhatsApp conversation for Lead L1018 with Diana...

Simulating WhatsApp conversation for Lead L1019 with Alice...

Simulating WhatsApp conversation for Lead L1022 with Bob...

Simulating WhatsApp conversation for Lead L1024 with Alice...

Simulating WhatsApp conversation for Lead L1027 with Bob...

Simulating WhatsApp conversation for Lead L1028 with Alice...
